# Score de pagamentos Itau Cartões A03

Esse notebook tem por objetivo criar um modelo de classificação de probabilidade de pagamentos para o segmento A03 da carteira Itau Cartões da Zanc.
Faremos uso de uma base de cerca de 100 mil cpfs que tiveram permanencia de 45 completos dentro da empresa.
Nosso target é conseguir distinguir quem tem mais chances de recuperação de crédito de quem não tem.

Hoje já obtivemos algum resultado utilizando Logistic Regression através da ferramenta Orange.
A tarefa inicial será de transportar para esse notebookk com código python o que foi feito no Orange.
Na seqüência será proposto uma outra abordagem utilizando Gradient Boosting

## Preparação dos dados

#### Importando módulos necessários

In [51]:
import numpy as np # Sem ele não somos ninguém
import time # Para medir o tempo de execução dos modelos
import pandas as pd # Para carregar os dados em um datafram
from sklearn.utils import resample # Para balancear os dados
from sklearn.impute import SimpleImputer # Para preencher os campos que estivem com valores nulos
from sklearn.preprocessing import MinMaxScaler # Para normalização dos campos numéricos
from sklearn.linear_model import LogisticRegression # Para instanciar o modelo de Regressão Logística
from sklearn.ensemble import GradientBoostingClassifier # Para instanciar o modelo de Gradient Boosting
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV # Para avaliação dos modelos
import pickle # Para salvar o modelo para uso futuro
import joblib # Para salvar o modelo para uso futuro
from Tratamentos import converte_publico, trata_loja

#### Importando os dados para um df pandas

Vamos importar o dataframe utilizando a funcao read csv padrão do pandas, analisando as colunas, tipos e valores nulos.

In [45]:
df = pd.read_csv(r"data\prever_novo.csv", encoding="latin1", delimiter=";", decimal = ",")
df.head()

,LOJA,renda,cpf_cnpj,scorecontratante,dataentrada,validadecampanha,atrasocongelado,valorcartacampanha,vlclusters,status_boletagem,data_status_boletagem,desconto,bandeira,publico,matriz,Pagamentos,Acionamentos,Valor Pago
0,CARTÃO PRIV LBL FIC ASSAI,0,74713,425,21/02/2019 00:00,17/04/2019,253,1582.10,1894.25,BOLETAR_A_PARTIR_,17/03/2019,50.72,FC,0,3,0,26,NaN
1,CARTÃO EXTRA 2.0,0,333751,428,25/04/2019 00:00,19/06/2019,192,392.90,439.42,BOLETAR_A_PARTIR_,19/05/2019,33.60,FC,0,4,0,0,NaN
2,MAGAZINE LUIZA/LUIZACRED FLEX,0,676527,343,02/03/2019 00:00,26/04/2019,197,1069.87,1353.72,BOLETAR_A_PARTIR_,26/03/2019,42.52,LC,0,3,0,5,NaN
3,CARTÃO EXTRA 2.0,0,919799,353,13/12/2018 00:00,06/02/2019,369,1002.21,1717.83,BOLETAR_A_PARTIR_,06/01/2019,71.04,FC,0,3,0,7,NaN
4,CARTÃO EXTRA 2.0,0,1140140,371,18/04/2019 00:00,12/06/2019,307,874.41,988.66,BOLETAR_A_PARTIR_,12/05/2019,53.04,FC,ElegÃ­vel ExceÃ§Ã£o,3,0,1,NaN


In [6]:
df.shape

(111420, 15)

In [6]:
df.dtypes

LOJA                      object
renda                      int64
cpf_cnpj                   int64
scorecontratante           int64
dataentrada               object
validadecampanha          object
atrasocongelado            int64
valorcartacampanha       float64
vlclusters               float64
status_boletagem          object
data_status_boletagem     object
desconto                 float64
bandeira                  object
publico                   object
matriz                     int64
Pagamentos                 int64
Acionamentos               int64
Valor Pago                object
dtype: object

In [7]:
df.isna().sum()

LOJA                          0
renda                         0
cpf_cnpj                      0
scorecontratante              0
dataentrada                   0
validadecampanha              0
atrasocongelado               0
valorcartacampanha            0
vlclusters                    0
status_boletagem              0
data_status_boletagem         0
desconto                      0
bandeira                      0
publico                       0
matriz                        0
Pagamentos                    0
Acionamentos                  0
Valor Pago               109295
dtype: int64

#### Normalizando campos numéricos

In [46]:
escalador_vlrcartacampanha = MinMaxScaler()
escalador_vlrccluster = MinMaxScaler()
escalador_vlrdesconto = MinMaxScaler()
df['valorcartacampanha_s'] = escalador_vlrcartacampanha.fit_transform(df[['valorcartacampanha']].values)
df['vlclusters_s'] = escalador_vlrccluster.fit_transform(df[['vlclusters']].values)
df['desconto_s'] = escalador_vlrdesconto.fit_transform(df[['desconto']].values)

#### Tratando Loja

In [54]:
def trata_loja(x):
    lojas = ['CARTÃO EXTRA 2.0',
             'CARTÃO MARISA 2.0',
             'CARTAO PL EMBANDEIRADO MARISA',
             'CARTAO PL FIC CB S/P',
             'CARTAO PL FIC EXTRA BAND',
             'CARTAO PL FIC EXTRA S/P',
             'CARTÃO PONTO FRIO 2.0',
             'CARTÃO PRIV LBL FIC ASSAI',
             'CARTÃO WALMART 2.0',
             'CREDICARD CLASSICOS',
             'HIPERCARD',
             'ITAUCARD 2.0 CANAIS DIRETOS',
             'MAGAZINE LUIZA/LUIZACRED FLEX',
             'OPERACOES CREDITO CREDICARD',
             'OUTROS',
             'TAM ITAUCARD 2.0']
    if x in lojas:
        return x
    else:
        return 'OUTROS'
df['LOJA'] = df['LOJA'].apply(trata_loja)
df['LOJA'].value_counts()

OUTROS                           44677
MAGAZINE LUIZA/LUIZACRED FLEX    32317
HIPERCARD                        14884
CREDICARD CLASSICOS               4567
CARTAO PL FIC CB S/P              3402
ITAUCARD 2.0 CANAIS DIRETOS       3080
OPERACOES CREDITO CREDICARD       2721
CARTAO PL FIC EXTRA S/P           2154
TAM ITAUCARD 2.0                  1450
CARTAO PL EMBANDEIRADO MARISA     1435
CARTAO PL FIC EXTRA BAND          1161
Name: LOJA, dtype: int64

#### Tratando público

In [48]:
df['publico'].value_counts()

0                         70152
elegivel_excecao          23983
ElegÃ­vel ExceÃ§Ã£o       11902
Elegivel Excecao           2639
alto_atrito                1939
Alto Atrito                1090
Eleg?Â¡vel Exce?Âº?Ãºo      142
883000140672                  1
Name: publico, dtype: int64

In [53]:
def converte_publico(x):
    if x=="Eleg?¡vel Exce?º?úo" or x=="Elegível Exceção" or x=="Elegivel Excecao" or x=="ElegÃ­vel ExceÃ§Ã£o" or x=="Eleg?Â¡vel Exce?Âº?Ãºo":
        return "elegivel_excecao"
    elif x=="Alto Atrito":
        return "alto_atrito"
    elif x=="0" or x=="883000140672":
        return "nao_definido"
    else:
        return x

df['publico'] = df['publico'].apply(converte_publico)
df['publico'].value_counts()

nao_definido        70153
elegivel_excecao    38666
alto_atrito          3029
Name: publico, dtype: int64

#### Transformando colunas categóricas em binárias

In [55]:
colunas_treino = ['LOJA',
                  'renda',
                  'scorecontratante',
                  'atrasocongelado',
                  'valorcartacampanha_s',
                  'vlclusters_s',
                  'status_boletagem',
                  'desconto_s',
                  'bandeira',
                  'publico',
                  'matriz'
                 ]

In [56]:
dummies = pd.get_dummies(df[colunas_treino])
dropado = df.drop(colunas_treino,axis=1)
df = pd.concat([dummies,dropado], axis=1)

#### Separando treino e teste

In [11]:
df['Pagamentos'].value_counts()

0    109295
1      2553
Name: Pagamentos, dtype: int64

In [57]:
df_maioria = df[df.Pagamentos==0]
df_minoria = df[df.Pagamentos==1]
df_maioria_randomizado = resample(df_maioria, 
                                 replace=True,     
                                 n_samples=2533,    
                                 random_state=123)
df_balanceado = pd.concat([df_maioria_randomizado, df_minoria])
df_balanceado.Pagamentos.value_counts()

1    2553
0    2533
Name: Pagamentos, dtype: int64

In [38]:
dummies.columns

Index(['renda', 'scorecontratante', 'atrasocongelado', 'valorcartacampanha_s',
       'vlclusters_s', 'desconto_s', 'matriz',
       'LOJA_CARTAO PL EMBANDEIRADO MARISA', 'LOJA_CARTAO PL FIC CB S/P',
       'LOJA_CARTAO PL FIC EXTRA BAND', 'LOJA_CARTAO PL FIC EXTRA S/P',
       'LOJA_CREDICARD CLASSICOS', 'LOJA_HIPERCARD',
       'LOJA_ITAUCARD 2.0 CANAIS DIRETOS',
       'LOJA_MAGAZINE LUIZA/LUIZACRED FLEX',
       'LOJA_OPERACOES CREDITO CREDICARD', 'LOJA_OUTROS',
       'LOJA_TAM ITAUCARD 2.0', 'status_boletagem_0',
       'status_boletagem_BOLETAR_A_PARTIR_',
       'status_boletagem_BOLETAR_A_VONTADE', 'bandeira_CC', 'bandeira_CR',
       'bandeira_FA', 'bandeira_FC', 'bandeira_FT', 'bandeira_HC',
       'bandeira_LC', 'bandeira_MA', 'publico_alto_atrito',
       'publico_elegivel_excecao', 'publico_nao_definido'],
      dtype='object')

In [58]:
y_train = df_balanceado.Pagamentos.values
X_train = df_balanceado[dummies.columns].values
y_test = df.Pagamentos.values
X_test = df[dummies.columns].values

#### Treinando e predizendo com LogReg


In [59]:
param_grid_lr = {
    'dual': [True,False],
    'max_iter': [100,110,120,130,140],
    'C': [1.0,1.5,2.0,2.5,3.5]
}
lr = LogisticRegression()
grid_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=5, scoring='roc_auc', n_jobs=-1)
start_time = time.time()
grid_result = grid_lr.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Best: 0.649833 using {'C': 3.5, 'dual': False, 'max_iter': 100}
Execution time: 6.564402103424072 ms


C:\Users\joaoc\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [281]:
pred = grid_result.predict_proba(X_test)
df_predicted = df_dummies
df_predicted['pagamentos real'] = y_test
df_predicted['prob prevista'] = pred[:,1]
df_predicted.to_csv('logreg_predicted.csv', decimal=',', sep=";")

#### Treinando e predizendo com XgBoosting

In [60]:
param_grid_xg = {
        'n_estimators':[100,200,400,600],
        'min_samples_leaf':[5,10,20],
        'subsample': [0.6, 0.8, 1.0]
        }
xg = GradientBoostingClassifier(learning_rate=0.02)
grid_xg = GridSearchCV(estimator=xg, param_grid=param_grid_xg, cv=5, scoring='roc_auc', n_jobs=-1)
grid_result_xg = grid_xg.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result_xg.best_score_, grid_result_xg.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Best: 0.679539 using {'min_samples_leaf': 5, 'n_estimators': 200, 'subsample': 0.6}
Execution time: 150.36838054656982 ms


In [40]:
pred = grid_result_xg.predict_proba(X_test)
df_predicted = df
df_predicted['prob prevista'] = pred[:,1]
df_predicted.to_csv(r'data\xgboost_train_result_predict.csv', decimal=',', sep=";")

#### Salvando os Scalers e o Xgboosting

In [61]:
joblib.dump(escalador_vlrcartacampanha, r"models\scaler_vlrcartacampanha_a03.pkl")
joblib.dump(escalador_vlrccluster, r"models\scaler_vlrcluster_a03.pkl")
joblib.dump(escalador_vlrdesconto, r"models\scaler_vlrdesconto_a03.pkl")
joblib.dump(grid_result_xg, r"models\xgboosting_a03.pkl")

['models\\xgboosting_a03.pkl']

 #### To-do

1. ~~Antes de gerar as dummies montar uma transformação para a coluna publico corrigindo: 'publico_Alto Atrito', 'publico_Eleg?¡vel Exce?º?úo', 'publico_Elegivel Excecao', 'publico_Elegível Exceção','publico_Não definido', 'publico_alto_atrito','publico_elegivel_excecao'~~
2. ~~Ajutar a métrica para ROC~~
3. ~~Rodar GridSearch para os hiperparâmetros da regLog e Xgboost otimizando ROC~~
4. ~~Selecionar o modelo e montar .py para predizer através do DW~~
5. Integrar com DW para criar uma coluna com a probabilidade prevista
6. Criar tratamento para nulos, pois eles foram retirados nessa amostra e será necessário para treinar no futuro
